In [10]:
'''
1. 필요 패키지 불러오기
2. 데이터 불러오기
3. 4월과 7월의 해당 AMT들을 3월의 AMT + Random Noise로 대체
4. 3월에 해당 AMT가 없을 경우 3월 이전의 가장 최신값으로 대체
5. csv 변환
'''

'\n1. 필요 패키지 불러오기\n2. 데이터 불러오기\n3. 4월과 7월의 해당 AMT들을 3월의 AMT + Random Noise로 대체\n4. 3월에 해당 AMT가 없을 경우 3월 이전의 가장 최신값으로 대체\n5. csv 변환\n'

## 필요 패키지 불러오기

In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import font_manager,rc
import matplotlib

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib inline

print ('버전: ', mpl.__version__)
print ('설치 위치: ', mpl.__file__)
print ('설정 위치: ', mpl.get_configdir())
print ('캐시 위치: ', mpl.get_cachedir())

print ('설정파일 위치: ', mpl.matplotlib_fname())

font_list_mac = fm.OSXInstalledFonts()
print(len(font_list_mac))

[(f.name, f.fname) for f in fm.fontManager.ttflist if 'Gothic' in f.name]

버전:  3.2.2
설치 위치:  /opt/anaconda3/envs/venv1/lib/python3.7/site-packages/matplotlib/__init__.py
설정 위치:  /Users/keom/.matplotlib
캐시 위치:  /Users/keom/.matplotlib
설정파일 위치:  /opt/anaconda3/envs/venv1/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc
362


/opt/anaconda3/envs/venv1/lib/python3.7/site-packages/ipykernel_launcher.py:23: MatplotlibDeprecationWarning: 
The OSXInstalledFonts function was deprecated in Matplotlib 3.1 and will be removed in 3.3.


[('Noto Sans Gothic',
  '/System/Library/Fonts/Supplemental/NotoSansGothic-Regular.ttf'),
 ('Apple SD Gothic Neo', '/System/Library/Fonts/AppleSDGothicNeo.ttc'),
 ('AppleGothic', '/System/Library/Fonts/Supplemental/AppleGothic.ttf'),
 ('Hiragino Maru Gothic Pro', '/System/Library/Fonts/ヒラギノ丸ゴ ProN W4.ttc')]

## Importing

In [3]:
os.listdir('jeju_data_ver1')

data = pd.read_csv('./jeju_data_ver1/201901-202003.csv')
sub = pd.read_csv('./jeju_data_ver1/submission.csv')

data.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


In [4]:
df = data[['REG_YYMM', "CARD_SIDO_NM", "STD_CLSS_NM", 'AMT']].groupby(
    ['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

print(df.shape)
print(df.head())

(9438, 4)
   REG_YYMM CARD_SIDO_NM        STD_CLSS_NM         AMT
0    201901           강원         건강보조식품 소매업   148354746
1    201901           강원            골프장 운영업  1229200312
2    201901           강원        과실 및 채소 소매업  1286979106
3    201901           강원  관광 민예품 및 선물용품 소매업    30656770
4    201901           강원    그외 기타 스포츠시설 운영업    13101500


### 2020년 3월에 값이 있으면 4월과 7월의 AMT열을 3월 AMT에 random noise 더한 값으로 대체

In [5]:
df_03 = df[df["REG_YYMM"]==202003]
df_03.shape

sigma, mu = 1.5, 0.

for i in np.arange(0, df_03.shape[0]):
    for j in np.arange(0, sub.shape[0]):
        if df_03.iloc[i, 1] == sub.iloc[j, 2] and df_03.iloc[i, 2] == sub.iloc[j, 3]:
            sub.iloc[j, 4] = df_03.iloc[i, 3] + sigma*np.random.randn(1)

### 2020년 3월에 값이 없으면 가장 3월 이전의 가장 최신 값으로 대체

In [6]:
df_rev = df[::-1]

for j in np.arange(0, sub.shape[0]):
    for i in np.arange(0, df_rev.shape[0]):
        if df_rev.iloc[i, 1] == sub.iloc[j, 2] and df_rev.iloc[i, 2] == sub.iloc[j, 3]:
            sub.iloc[j, 4] = df_rev.iloc[i, 3]+ sigma*np.random.randn(1)

### 2020년 4월/7월에 특정 지역에 특정 산업이 새롭게 등장하는 경우가 94건

In [7]:
(sub['AMT'] == 0).sum()

94

In [8]:
sub[sub['AMT']==0].head()

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
30,30,202004,강원,정기 항공 운송업,0.0
55,55,202004,경기,면세점,0.0
71,71,202004,경기,정기 항공 운송업,0.0
96,96,202004,경남,면세점,0.0
112,112,202004,경남,정기 항공 운송업,0.0


In [9]:
df[(df['CARD_SIDO_NM'] == '강원') & (df['STD_CLSS_NM'] == '정기 항공 운송업')]

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT


## 0인 자료들 min으로 대체하는게 더 부정확하다

```python
for i in np.arange(0, sub.shape[0]):
    if sub.iloc[i, 4] == 0.:
        sub.iloc[i,4] = df[df['STD_CLSS_NM'] == sub.iloc[i, 3]].min()[3]

(sub['AMT'] == 0).sum()
```

In [50]:
sub = pd.DataFrame(sub, index = sub['id']).drop(['id'], axis=1)

sub.to_csv('model_random_walk.csv', encoding='utf-8-sig')